In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Для ускорения apply функций данные немаленькие
from pandarallel import pandarallel

pandarallel.initialize()

sns.set()

%config InlineBackend.figure_format = 'svg'

plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["image.cmap"] = "viridis"

# Загрузка данных

In [ ]:
data = pd.read_parquet("../data/transaction_fraud_data.parquet")
exchange = pd.read_parquet("../data/historical_currency_exchange.parquet")

In [ ]:
data = pd.concat((data, pd.json_normalize(data["last_hour_activity"])), axis=1).drop(
    columns="last_hour_activity"
)

In [ ]:
exchange["date"] = pd.to_datetime(exchange["date"]).dt.date
data["date"] = data["timestamp"].dt.date

# EDA

In [ ]:
data.head()

In [ ]:
exchange.head()

In [ ]:
data.info()

In [ ]:
exchange.info()

In [ ]:
data = data.merge(exchange, how="left", on="date")

In [ ]:
data.isna().sum()

In [ ]:
data["is_fraud"].value_counts(normalize=True)

Сразу конвертируем amount в USD

In [ ]:
data["usd_amount"] = data.parallel_apply(
    lambda x: x["amount"] / x[x["currency"]], axis=1
)
data["usd_total_amount"] = data.parallel_apply(
    lambda x: x["total_amount"] / x[x["currency"]], axis=1
)
data["usd_max_single_amount"] = data.parallel_apply(
    lambda x: x["max_single_amount"] / x[x["currency"]], axis=1
)

Посмотрим распределение usd_amount для is_fraud

In [ ]:
sns.boxplot(data=data, y="usd_amount", x="is_fraud");